# Introduction to MuSpAn for Quantitative Spatial Analysis: MSc Genomic Medicine

This workshop is designed to introduce you to the fundamental concepts and applications of MuSpAn, helping you develop pipelines for spatial data analysis in biomedical research.

Understanding the spatial organisation of cells and molecules is crucial in genomic medicine, particularly for studying disease progression and therapeutic response. This session will provide hands-on experience with MuSpAn, equipping you with the skills to analyse spatial data in a meaningful way.

---

### What to Expect
This practical will be divided into two (three) key sections:

1. Introduction – Learn the basics of MuSpAn, its core functionalities, and how to set up your first spatial analysis pipeline.
2. Application – Apply MuSpAn to spatially analyse immune-stromal interactions in colorectal cancer, gaining insights into spatial quantification techniques.
3. Extension (time permitting) – Explore advanced applications, including spatial transcriptomics imaging data to uncover gene expression patterns in tissue samples.

By the end of this session, you'll have a solid foundation in spatial data analysis using MuSpAn, ready to apply these techniques in your research. 

---

### Resources

Throughout this session, we highly encourage you to use the supporting resources (documentation, tutorial, paper examples, glossary) for MuSpAn which can be found at https://www.muspan.co.uk. The reference paper for this session: https://www.biorxiv.org/content/10.1101/2024.12.06.627195v2

Let’s get started!

---

## 1. Introduction: Getting started with MuSpAn

This section will cover the basics in our tutorial series 'Getting Started' which can be found at https://docs.muspan.co.uk/latest/tutorials.html. 


In this section we will:
- Create a MuSpAn Domain
- Add spatial data (with annotations) to this domain from a csv file 
- Visualise our data
- Generate the boundary of our data
- Analyse the spatial distribution of immune cells in a sample of mouse colorectal cancer


---

### Installing MuSpAn

If you haven't already, we can install MuSpAn like any other Python library using the pip package manager. See https://docs.muspan.co.uk/latest/Installation.html for more details.

Once you have followed the necessary steps, you should have MuSpAn ready for spatial analysis. Let's check this.

In [ ]:
import muspan as ms
print('MuSpAn was successfully imported!')

### An introductory walkthrough analysis 

In order to access the spatial analysis tools provided by MuSpAn, it needs to know what your data looks like. In this tutorial, we introduce the concept of a <i>domain</i>, the central object of interest within MuSpAn. You can think of the domain as being a container holding all the information relevant to a single spatial frame of reference, such as a region of interest. Let's make a new domain now, by calling ```ms.domain()```:

In [ ]:
# Create a new domain with the name 'Example'
my_domain = ms.domain('Example')

#print out the domain
print(my_domain)

This domain is pretty boring at the moment. ```ms.domain()``` requires us to give the domain a name (useful for when we're working with multiple regions of interest at once), here 'Example'. Calling ```print(my_domain)``` tells us about the contents of the domain, but at the moment there's nothing there. We need to populate our domain with some data.

Spatial data is stored in MuSpAn as <i>objects</i>: each object can be a <i>point</i>, <i>line</i>, or <i>shape</i>. These objects can represent anything we'd like. For example, we could have points representing cells, transcripts, or, shapes representing cellular boundaries, tissue regions of interest annotated in other software such as QuPath. It all depends on our interpretation of that object - the software doesn't care!

Domains are not static objects, they update as we interact with them through our analysis. For example, we can add and delete objects. Labels can be added from predefined annotations or generated through a specific spatial method. That is to say, our domain is an editable *bucket* of spatial data.

To see this in action, let's get a spatial dataset of a sample of murine colorectal carcinoma stored in the ```muspan.datasets``` submodule. This dataset is stored in a csv file which is a typical format for this type of data following image segmentation and annotation. The libraries ```pandas``` and ```os``` useful to handle data and file management - we'll import these to load in our csv file.

This sample was preprocessed using Halo. Cells were segmented and labelled using range of immune and epithelial markers. For more information on this dataset, see https://doi.org/10.1017/S2633903X24000011.

In [ ]:
import pandas as pd
import os

# Load the dataset
# The dataset is stored within MuSpAn for use as an example
# We read the CSV file into a pandas DataFrame
df = pd.read_csv(os.path.dirname(ms.datasets.__file__) + '/data/Bull_2024_mouse_colon.csv')

# Display the first 5 rows of the dataframe to understand its structure
print(df)


In this dataset, we have:
1. segmented cells represented by their centroid which are define by their (x,y) coordinates,
2. a phenotypic classification given in the 'Cell type' column,
3. the mean channel intensity of the CD4 marker for all cells.


To start, we'll just take a focus on the loading in the cells as points.

In [ ]:
# Import numpy library
import numpy as np

# Extract 'x' and 'y' columns from the dataframe and convert them to a numpy array
points = np.asarray([df['x'], df['y']])

# Print the shape of the resulting numpy array
print(np.shape(points))

Note that stacking the x and y columns together has generated a (2 x 6676) numpy array. In order to add this to a MuSpAn domain, we need it to be (n x 2) dimensional, so we’ll have to transpose the array to add the points. Let’s make a MuSpAn domain, add the (transposed) array of points.

In [5]:
# Add the transposed array of points to the domain with the label 'Cell centres'
my_domain.add_points(points.T, 'Cell centres')

Let's take a look to make sure our cells are correctly loaded in. We can visualise our domain using the ```muspan.visualise``` module. MuSpAn isn't intended to be a visualisation tool, but it's always handy to be able to see our data. MuSpAn uses [matplotlib](https://matplotlib.org/) to provide some handy tools to see your data quickly, so we'll import that too. Let's take a look at our points, using the ```ms.visualise.visualise()``` command:

In [ ]:
# Import matplotlib for plotting
import matplotlib.pyplot as plt


# Create a figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain
ms.visualise.visualise(my_domain, ax=plt.gca())

Metadata can be linked with MuSpAn objects using <i>labels</i>. In this case, let's add a label to each cell centre describing its cell type. Adding labels is a straightforward process, we just call ```domain.add_labels('Celltype',celltype_array)```, to add an array of labels with the name "Celltype" to the domain.

In [ ]:
# Extract the 'Cell type' column from the dataframe
celltype_array = df['Cell type']

# Add the cell type labels to the domain
my_domain.add_labels('Celltype', celltype_array)

# Print the domain to verify the labels have been added
print(my_domain)

We can see that the list of available Labels now contains 'Celltype'.

If we want to see what these labels look like, we can add an argument to the command ```ms.visualise.visualise(domain, 'Celltype')```, which MuSpAn will interpret as wanting to colour the objects in ```domain``` using a label called ```celltype```:

In [ ]:
# Visualise the domain, colouring objects by the 'Celltype' label
ms.visualise.visualise(my_domain, 'Celltype')

By default, MuSpAn will try to select a sensible colour palette for visualisation, but there are many ways to customise visualisation in MuSpAn (see the tutorials on visualisation for examples). For now, let's manually change some of the colors associated with this label to something with a bit less orange in it. We can do that by passing a dictionary of the colours we'd like to update to ```domain.update_colors```. Let's make the epithelium grey, and use the orange colour for T Helper Cells. MuSpAn uses ```pyplot``` for plotting, so we can specify colours in any way that ```matplotlib``` recognises. While we're here, we'll also use slightly smaller markers so that nearby points don't overlap.

In [ ]:
# Define new colors for specific cell types
new_colors = {'Epithelium': 'grey', 'T Helper Cell': plt.cm.tab10(1)}

# Update the colors in the domain for the 'Celltype' label
my_domain.update_colors(new_colors, label_name='Celltype')

# Create a new figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain, colouring objects by the 'Celltype' label with updated colors and smaller markers
ms.visualise.visualise(my_domain, 'Celltype', ax=plt.gca(), marker_size=5)

We may also want to show just certain cell types, for instance, only the Regulatory T Cells. We can do that by passing a tuple ```(label_name,label_value)``` as the keyword argument ```objects_to_plot```.

In [ ]:
# Create a new figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain, showing only the 'Regulatory T Cell' type
# The 'show_boundary' argument is set to True to display the boundary of the domain
ms.visualise.visualise(my_domain, 'Celltype', objects_to_plot=('Celltype', 'Regulatory T Cell'), ax=plt.gca(), show_boundary=True)

'Celltype' here is an example of a <i>categorical</i> label, in which the label takes distinct values. We can also assign <i>continuous</i> labels, such as the 'CD4 intensity' label included with this dataset (which describes the average expression of the marker CD4, found on T Helper Cells).

In [ ]:
# Extract the 'CD4 intensity' column from the dataframe
CD4_intensity = df['CD4 intensity']

# Add the CD4 intensity labels to the domain with a colormap 'viridis'
my_domain.add_labels('CD4', CD4_intensity, cmap='viridis')

# Create a new figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain, colouring objects by the 'CD4' label with smaller markers
ms.visualise.visualise(my_domain, 'CD4', ax=plt.gca(), marker_size=5)

A crucial part of the domain is the <i>boundary</i> - in order to do spatial analysis, we need to know not just what objects we're analysing, but also what space they live in! Just because we haven't specified a boundary doesn't mean that it isn't there. MuSpAn automatically infers a domain boundary whenever we make a domain, or add new data. By default, this boundary will be the minimal axis aligned bounding box for the data. The boundary is a special type of <i>shape</i> object - it can be accessed easily via ```my_domain.boundary```, e.g.:

In [ ]:
print(my_domain.boundary)

The boundary always has ID -1, and in this case we can see that it is formed from 4 *vertices*. Every object in MuSpAn is made of vertices. These are just 2D coordinates that define the object. Every point is made of 1 vertex, a line is an ordered of vertices, and a shape is an ordered list of vertices that finishes where it started (note that MuSpAn has other requirements for shapes and lines, such as not being self-intersecting, but let's not worry about that for now).

Our boundary shape has 4 vertices - where are they? By default, ```ms.visualise.visualise()``` doesn't show the domain boundary, but we can easily turn it on by passing the argument ```show_boundary=True```.

In [ ]:
# Create a new figure with a specified size
plt.figure(figsize=(8, 5))

# Visualise the domain using MuSpAn's visualisation tool
# This will plot the points we added to the domain and show the boundary
ms.visualise.visualise(my_domain,color_by='Celltype', ax=plt.gca(), show_boundary=True,marker_size=5)

# Print the area of the domain boundary
print(f'Domain area: {my_domain.boundary.area}')

Great, now we know our boundary is appropriate for our data, we can now start to think about our spatial analysis!

A good summative method for quantifying which pair of cell types are co-located is a region-based method called the Quadrat Correlation Matrix (QCM). The QCM works by subdividing our domain into smaller regions and testing for significance of celltypes found within the same region through spatial permutation. Think of it as a way to check how often certain things (like points with specific labels) tend to show up in the same neighborhood—essentially looking for patterns of “hanging out together” in space.

Let's run this using the `quadrat_correlation_matrix` function, found in the `region_based` module, to perform QCM analysis.

In [ ]:
# Perform region-based quadrat correlation matrix analysis on the domain
# SES: Standardized Effect Size matrix
# A: Observed correlation matrix
# resulting_labels: Labels used in the analysis
SES, A, resulting_labels = ms.region_based.quadrat_correlation_matrix(
    my_domain,                      # The domain to analyse
    label_name='Celltype',          # The label to use for the analysis
    region_method='quadrats',       # The method to define regions (quadrats in this case)
    region_kwargs=dict(side_length=75),  # Arguments for the region method (side length of quadrats)
    low_observation_bound=5,        # Minimum number of observations required in a region
    visualise_output=True           # Whether to visualise the output
)

This has produced a pair-wise summary the spatial localisation observed in our domain. But our interpretation will dependent on the context of our spatial data, consider the following questions below:

**Questions**:
1. What is the interpretation of this figure?
2. How does the side_length impact the results and interpretation? 
3. What are the assumptions of the QCM?
4. Which of these interactions are statistically significant?

(Add notebook cells above or below to help answer these questions with some code).

Take a look at the Quadrat correlation analysis tutorial at https://docs.muspan.co.uk/latest/_collections/region_based_analysis/quadrat_correlation.html and the documentation https://docs.muspan.co.uk/latest/muspan.region_based.html to help with below questions.

**Answers**:
1. ...
2. ...
3. ...
4. ...

---

Following the identification of co-localised of specific cell types in domain, we may want analysis their spatial distribution of mulitple spatial scales. One of the many ways of doing this in MuSpAn is to use the cross Pair Correlation Function (cross-PCF). 

The cross-PCF [(also known as the Radial Distribution Function)](https://en.wikipedia.org/wiki/Radial_distribution_function) is a spatial statistic that characterises <i>clustering</i> or <i>exclusion</i> at different length scales. Consider two populations of points, that we'll call $A$ and $B$. Then the cross-PCF, $g_{AB}(r)$, can be thought of as a ratio describing whether the observed number of pairs of points separated by distance $r$ (where one is type $A$ and the other type $B$) is higher or lower than would be expected under a statistical null model. In brief, $g_{AB}(r) > 1$ indicates increased numbers of $A-B$ pairs separated by distance $r$, which indicates clustering. $g_{AB}(r) < 1$ suggests exclusion, or regularity.

We demonstrate the use of the cross PCF this on our domain by looking at the interactions between 'T Helper Cells' using the ```cross_pair_correlation_function``` in the ```muspan.spatial_statistics``` module. As before, we can visualise the results of this statistic using the `visualise_output` parameter.

In [ ]:
# Perform cross pair correlation function analysis on the domain
# This function calculates the cross-PCF between two populations of points
# population_A and population_B are the two populations to analyze
# max_R is the maximum radius to consider for the analysis
# annulus_width is the width of the annuli used in the analysis
# annulus_step is the step size between annuli
# visualise_output=True will generate a visualisation of the results
r, g = ms.spatial_statistics.cross_pair_correlation_function(
    my_domain,
    population_A=('Celltype', 'T Helper Cell'),
    population_B=('Celltype', 'T Helper Cell'),
    max_R=350,
    annulus_width=25,
    annulus_step=10,
    visualise_output=True
)

**Questions**:
1. What is the interpretation of this figure?
2. How does the annulus width and annulus step impact the results and interpretation? 
3. What are the assumptions of the cross-PCF?

(Add notebook cells above or below to help answer these questions with some code).

Use the cross-PCF tutorial (https://docs.muspan.co.uk/latest/_collections/spatial_analysis_methods/Spatial%20stats%20-%201%20-%20pcf.html) and documentation (https://docs.muspan.co.uk/latest/generated/muspan.spatial_statistics.cross_pair_correlation_function.html#muspan.spatial_statistics.cross_pair_correlation_function) to help answer these questions.

**Answers**:
1. ...
2. ...
3. ...

---

**Task:**

Using what we have learned from our QCM analysis, compute the cross-PCF for all significant pair-wise interactions to investigate the long range spatial relationships.

Tip: take a look at the cross-PCF tutorial for using cross-PCF function in loops.

In [16]:
## Your code here 






## 2. Application: Immune-stromal spatial relationships in colorectal cancer

Please pull from the repo again when you get to here!

In [17]:
## some code ...

## 3. Extension: Spatial analysis of spatial transcriptomics data

Spatial transcriptomics (ST) is an advanced genomic technology that enables the simultaneous measurement of gene expression and spatial localisation within tissue samples. Unlike traditional bulk or single-cell RNA sequencing, which disrupts tissue architecture, ST preserves spatial context, allowing researchers to map gene activity to specific locations within a biological sample.

For more information on ST, see https://www.science.org/doi/10.1126/science.abq4964

In general ST can be used to define novel and precise cell phenotypic annotations (cell types) using subcellular information. With access to multiscale spatial information gives rise for more oppotunities for multiscale spatial analysis, for example, see our showcase in  https://www.biorxiv.org/content/10.1101/2024.12.06.627195v2.

---

In this section we invite you to explore the ST reference dataset of a sample of healthy mouse colon that is stored in MuSpAn (see https://docs.muspan.co.uk/latest/muspan.datasets.html). This dataset was used to produce Figures 2 and 3 in the MuSpAn paper. Using this as inspiration, use MuSpAn to develop a spatial pipeline to investigate the following questions:

1. Are there morphological differences between cell clusters? If so, what is the spatial distributions of similar cellular morphologies
2. What types of spatial structures are present?  
3. What is cells are directly in contact with Cluster 2 cells? Is this statisically significant?
4. What is the transcript composition of Cluster 9?


In [18]:
## Your code here